In [ ]:
pip install iduedu

In [ ]:
pip install pyarrow # To read .parquet

In [1]:
from iduedu import get_intermodal_graph
import geopandas as gpd
data = gpd.read_parquet('data/spb_buildings.parquet')
polygon = data.unary_union.convex_hull.buffer(0.001) # Getting an "encircling" polygon around our layer to download the graph 

In [2]:
G_intermodal = get_intermodal_graph(polygon =polygon, clip_by_bounds=True)

2024-10-30 21:31:22.279 | INFO     | iduedu.modules.drive_walk_builder:get_walk_graph:214 - Downloading walk graph from OSM, it may take a while for large territory ...


Parsing public transport routes:   0%|          | 0/51 [00:00<?, ?it/s]

Calculating the weights of the walk graph:   0%|          | 0/10076 [00:00<?, ?it/s]

2024-10-30 21:31:26.391 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:50 - Composing intermodal graph...
2024-10-30 21:31:26.670 | WARNING  | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:179 - Weakly connected components detected. 2 graph islands were removed. These are probably pt routes that are not connected to walking routes. You may try adjusting the max_dist parameter, but this can lead to incorrect data.


In [4]:
import numpy as np
from iduedu import get_adj_matrix_gdf_to_gdf

data.to_crs(G_intermodal.graph['crs'],inplace=True) # Have to use the same crs 

matrix = get_adj_matrix_gdf_to_gdf(data,data,G_intermodal,weight='time_min',threshold=50,dtype=np.float32) # Use dtype >np.float16 for more precision
matrix

,0,1,2,3,4,5,6,7,8,9,...,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257
0,0.00,7.77,16.360001,14.16,14.81,15.47,11.09,5.250000,3.25,1.37,...,16.809999,16.08,12.36,12.86,14.58,11.00,10.63,7.79,7.10,10.930000
1,7.22,0.00,15.490000,13.29,13.94,14.60,11.36,8.820000,6.82,6.65,...,15.940000,11.99,9.90,11.99,13.62,9.25,8.88,8.06,8.18,11.370000
2,15.33,14.08,0.000000,4.16,8.62,9.28,15.89,16.219999,14.22,14.49,...,10.200000,16.66,10.64,5.47,13.08,12.31,11.94,13.54,14.46,17.650000
3,13.70,11.88,4.160000,0.00,9.78,10.41,14.51,14.590000,12.59,12.86,...,11.320000,14.46,8.44,6.06,11.70,10.93,10.56,11.91,12.83,16.020000
4,13.73,11.45,8.620000,9.78,0.00,2.02,12.34,14.620000,12.62,12.89,...,3.380000,14.24,11.17,4.87,9.53,9.04,8.67,11.94,12.86,16.049999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,9.67,8.59,12.580000,10.38,9.50,10.16,6.89,10.560000,8.56,8.83,...,11.500000,12.30,8.58,7.55,6.28,0.00,0.37,7.75,8.80,11.990000
1254,9.30,8.22,12.210000,10.01,9.13,9.79,6.52,10.190000,8.19,8.46,...,11.130000,11.93,8.21,7.18,5.91,0.37,0.00,7.38,8.43,11.620000
1255,6.25,8.75,13.470000,11.27,11.92,12.58,7.10,7.140000,5.14,5.41,...,13.920000,13.19,9.47,9.97,12.12,8.11,7.74,0.00,5.38,8.570000
1256,8.18,10.12,14.110000,11.91,12.56,13.22,8.18,9.070000,7.07,7.34,...,14.560000,13.83,10.11,10.61,10.98,8.75,8.38,4.23,0.00,5.300000
